In [2]:
from collections import defaultdict
import itertools
from matplotlib import pyplot as plt
import math
import numpy as np
import operator
import pandas as pd
import pickle
import random
import re
import scipy
import string
from tqdm import tqdm_notebook


import src.knntools as knntools
import src.postprocess as postprocess
import src.preprocess as preprocess
import src.tfidftools as tfidftools
import src.tools as tools
import src.recencytools as recency
import src.scoring as scoring
import src.textembeddingtools as texttools
import src.graphwordstools as graphtools
import src.manualprocessingtools as manualtools

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
path_to_data = 'data/'

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)

training_info = pd.read_csv(
    path_to_data + 'training_info.csv', sep=',', parse_dates=True, header=0)

test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)

test_info = pd.read_csv(path_to_data + 'test_info.csv',
                        sep=',', parse_dates=True, header=0)
path_to_results = 'results/'

In [4]:
train_info, train_email_ids_per_sender, val_info, val_email_ids_per_sender = scoring.get_train_val(training, training_info, train_frac=0.95)

Processing training !

Processing val !



In [5]:
train_body_dict = preprocess.body_dict_from_panda(train_info)
val_body_dict = preprocess.body_dict_from_panda(val_info)

In [34]:
all_recipients_in_train = preprocess.get_all_recipients_from_df(train_info)
all_recipients_in_train = [rec.lower() for rec in all_recipients_in_train]

In [14]:
test_body_dict = preprocess.body_dict_from_panda(test_info)

In [43]:
test_email_ids_per_sender = preprocess.get_email_ids_per_sender(test)

In [44]:
mid_sender_dic = {}
for sender, mids in test_email_ids_per_sender.items():
    for mid in mids:
        mid_sender_dic[mid] = sender

In [47]:
emails_in_content = {}
pbar_emails = tqdm_notebook(test_body_dict.items())
for mid, body in pbar_emails:
    sender = mid_sender_dic[mid]
    found_emails = list(manualtools.get_emails(body))
    unique_found_emails = list(set(found_emails))
    # Keep only known recipients
    train_filtered_found_emails = [mail.lower() for mail in unique_found_emails if mail.lower() in all_recipients_in_train]
    # Remove sender email
    filtered_email = [mail for mail in train_filtered_found_emails if mail!=sender]
    if (filtered_email):
        emails_in_content[mid] = train_filtered_found_emails

In [48]:
emails_in_content
    

{1577: ['webmaster@cera.com'],
 30181: ['nidia.mendoza@enron.com'],
 31698: ['mark.evans@enron.com'],
 48412: ['eperrigo@epsa.org', 'paul.savage@nrgenergy.com'],
 48485: ['jhartso@enron.com'],
 48695: ['leslie.reeves@enron.com', 'wade.stubblefield@enron.com'],
 48901: ['acomnes@enron.com'],
 49096: ['jhartso@enron.com'],
 49264: ['jsteffe@enron.com'],
 51081: ['rrich@bracepatt.com'],
 51629: ['slmchargue@tecoenergy.com',
  'kbparker@tecoenergy.com',
  'jmwatson@tecoenergy.com'],
 53510: ['rrich@bracepatt.com'],
 71865: ['susan.jones@neg.pge.com'],
 80780: ['joanne.rozycki@enron.com'],
 82320: ['susan.jones@neg.pge.com'],
 85255: ['moshuffle@hotmail.com',
  'susan_m_scott@hotmail.com',
  'tkuykendal@enron.com'],
 85335: ['tkuyken@enron.com', 'drrichardson@cmsenergy.com'],
 85409: ['tkuyken@enron.com'],
 86785: ['mcuilla@houston.rr.com'],
 88834: ['physics123@yahoo.com'],
 100399: ['glaguarta@cadencemcshane.com'],
 111487: ['merharp@hotmail.com'],
 111944: ['susan_m_scott@hotmail.com'],


In [49]:
len(emails_in_content)

101

In [50]:
with open('variables/emails_in_test', 'wb') as outfile:
    pickle.dump(emails_in_content, outfile)